In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib
import math
import numpy as np
import random as rnd

colorNumber = 5
steps = 100000
loopRestart = 500
rng = np.random.default_rng(12345)# seed
isSolved = False

## Vygenerovani grafu do promenne G pomoci seedu nahody 

In [ ]:
def read_dimacs(filename):
    file = open(filename, 'r')
    lines = file.readlines()
    Gd = nx.Graph()
    for line in lines:
        if line[0] == "e":
            vs = [int(s) for s in line.split() if s.isdigit()]
            Gd.add_edge(vs[0]-1, vs[1]-1)
    return Gd

G = read_dimacs("DSJC125.1")

colorGraph = []
nodesCount = G.number_of_nodes()

## Nahodne vygenerovani barvy

In [ ]:
def randomColor(colorGraph):
    colorGraph = []
    for vrchol in range(nodesCount):
        colorNodes = rnd.randrange(0, colorNumber)
        colorGraph.append(colorNodes)
    return colorGraph
colorGraph = randomColor(colorGraph)

## Bere na vstupu pole barev vrcholu poporade, cislum priradi nahodne barvy a vykresli graf

In [ ]:
def plot(G, cols):
    k = np.max(cols)
    symbols = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F']
    colmap = ["#"+''.join(rng.choice(symbols, 6)) for i in range(k+1)]
    colors = [colmap[c] for c in cols]
    nx.draw(G, node_color=colors, with_labels=True)

## Testuje, jestli je konfigurace resenim

In [ ]:
def isColoring(graph, colors):
    result = True
    for nodeIndex, nameNodes in enumerate(graph):
        colorNodes = colors[nodeIndex]
        for numberOfNeighbor in G.neighbors(nameNodes):
            colorOfNeighbor = colors[numberOfNeighbor]
            if (colorNodes == colorOfNeighbor):
                result = False
    return result

## Vrati pocet kolizi v dane konfiguraci

In [ ]:
def collisions(graph, colors):
    collisions = 0
    edges = graph.edges()
    for edge in edges:
        n1, n2 = edge
        if colors[n1] == colors[n2]:
            collisions += 1
    return collisions

## Vygeneruje vsechny moznosti zmen v ramci zmeny jedne barvy

In [ ]:

def allOptionsOfColors(fieldOfColor):
    newFieldOfColors = []
    colorTemplate = set(fieldOfColor)
    for colorIndex, nameColor in enumerate(fieldOfColor):
        for uniqueColor in colorTemplate:
            temp = fieldOfColor.copy()
            if uniqueColor != nameColor: 
                temp[colorIndex] = uniqueColor
                newFieldOfColors.append(temp)
    return newFieldOfColors

## Vyhodnoceni vsech moznosti pole (k na nodes)

In [ ]:
def allOptions(fieldOfOptions):
    global isSolved
    lowestConflict = []
    for option in fieldOfOptions:
        numberOfCollisionOption = collisions(G, option)
        if numberOfCollisionOption == 0:
            isSolved = True
        lowestConflict.append(numberOfCollisionOption)
    return lowestConflict 

## Vystupem je jedna z nejlepsich moznosti

In [ ]:
def colorChanging(newOptions,completedOptions):
    sortedIndex = np.argsort(completedOptions) #vrati indexy serazeni
    #zjisti pocet moznosti stejne validity
    kCount = 0
    for k in completedOptions:
        if k == completedOptions[sortedIndex[0]]:
            kCount += 1
    if kCount > 1:
        #Vyber z moznosti, ktere jsou nejlepsi stejne
        nahodnyIndex = rnd.randrange(0, kCount)#Od nuly po n
        sortedIndex[0] = sortedIndex[nahodnyIndex]
    return newOptions[sortedIndex[0]]

## Hil Climbing Funkce 💥

In [ ]:
def color(G, colors, steps):
    global isSolved
    collisions = 0
    low = 999999 #def nejmensi pocet konfliktu na maxu
    minCount = 0 #Kolikrat jsem mel nejmensi pocet konfliktu
    tryed = 1 #Kolikrat jsem zkusil opakovat cely proces

    #Test na okamzite reseni == instantni reseni?
    if isColoring(G,colors):
        isSolved = True
    else:
    #pokud je nalezene reseni nebo dojde pocet kroku
        while (not isSolved and collisions < steps):
            if minCount >= loopRestart: #loopRestart po kolika stepech zresetuji cely graf pokud count bude n x stejna jako resetPromenna(200) tak zresetuji
                colors = randomColor(colors).copy() #vygeneruji novy pool barev a zresetuji promenne
                tryed += 1
                minCount = 0
                low = 999999              
            #Vygeneruje vsechny moznosti barev
            newFieldOfColors = allOptionsOfColors(colors)
            #Vyhodnoti colors list indexu nejmensich konfliktu
            completedOptions = allOptions(newFieldOfColors)  
            #Vystupem je jeden graf s nejmene konfliktami v "colors"
            colors = colorChanging(newFieldOfColors,completedOptions)
            #Zlepsil se pocet konfliktu?
            if completedOptions[0] < low:
                #low = nejmensi pocet konfliktu
                low = completedOptions[0]
                #reset low
                minCount = 0 #low collisions = pocet opakovani nejmensiho poctu konfliktu za sebou 8 8 = 1
            else:
                #Pokud se nezlepsuji, tak pricitam a zustavam na stejnem miste
                minCount += 1
            #Loguji pocet kroku s nejnizsim konfliktem a vidim jestli se zlepsuji
            print('Step: ' + str(collisions + 1) + ' Collisions: ' + str(low))
            collisions += 1 #Celkovy pocet vsech kroku

        if isSolved:
            print('Answer is completed on ' + str(tryed) + " try with " + str(colorNumber) + " colors.")
        else:
            print('Maybe next time')

        if collisions == 0:
            print('Graph was colored right before my operation')
        else:
            print('Final step is: ' + str(collisions))
        
    return colors, isSolved

In [ ]:
vysledek = color(G, colorGraph, steps)
plot(G, vysledek[0])#Vrati se mi pole ktere je true nebo false? vysledek[0] = barvy v returnu